In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.1.0


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [4]:
# Dataset Loading
train_dir = './data/best-artworks-of-all-time/images/images/'
test_dir = './data/best-artworks-of-all-time/images/testset/'

artist_names = os.listdir(train_dir)

artists = dict()
CLASS_WEIGHT = dict()

for artist_name in artist_names:
    artists[artist_name] = len(os.listdir(train_dir + artist_name))
artists = {k: v for k, v in sorted(artists.items(), key=lambda item: item[1] , reverse = True)}

for i , artist_name in enumerate(artists.keys()):
    CLASS_WEIGHT[i] =  max(artists.values())/artists[artist_name]
    artist_names[i] = artist_name


#sum(artists.values())/len(artists) # 예술가의 평균 그림 갯수
#CLASS_WEIGHT
print(artist_names)

['Vincent_van_Gogh', 'Edgar_Degas', 'Pablo_Picasso', 'Pierre-Auguste_Renoir', 'Albrecht_DuтХа├кrer', 'Paul_Gauguin', 'Francisco_Goya', 'Rembrandt', 'Alfred_Sisley', 'Titian', 'Marc_Chagall', 'Rene_Magritte', 'Amedeo_Modigliani', 'Paul_Klee', 'Henri_Matisse', 'Andy_Warhol', 'Mikhail_Vrubel', 'Sandro_Botticelli', 'Leonardo_da_Vinci', 'Peter_Paul_Rubens', 'Salvador_Dali', 'Hieronymus_Bosch', 'Pieter_Bruegel', 'Diego_Velazquez', 'Kazimir_Malevich', 'Giotto_di_Bondone', 'Frida_Kahlo', 'Gustav_Klimt', 'Raphael', 'Joan_Miro', 'Andrei_Rublev', 'Camille_Pissarro', 'Edouard_Manet', 'Vasiliy_Kandinskiy', 'El_Greco', 'Piet_Mondrian', 'Henri_de_Toulouse-Lautrec', 'Jan_van_Eyck', 'Claude_Monet', 'Henri_Rousseau', 'Diego_Rivera', 'Edvard_Munch', 'William_Turner', 'Gustave_Courbet', 'Caravaggio', 'Michelangelo', 'Paul_Cezanne', 'Georges_Seurat', 'Eugene_Delacroix', 'Jackson_Pollock']


In [17]:
# Configure the ImageDataGenerator
batch_size = 16 #increased batchsize from 32 to 128 
train_input_shape = (600, 600 , 3) # input shape for EfficientNet B-7
n_classes = len(artists)

my_strategy = tf.distribute.MirroredStrategy()
with my_strategy.scope():


    #Define Data Augmentation
    train_datagen = ImageDataGenerator(validation_split=0.15,
                                       rescale = 1./255.,   
                                       #featurewise_center = True,
                                       #featurewise_std_normalization = True
                                       #samplewise_center= True,
                                       #samplewise_std_normalization=True,
                                       zca_epsilon=1e-6,
                                       #zca_whitening = True,
                                       rotation_range=30,
                                       width_shift_range=0.3,
                                       height_shift_range=0.3,
                                       brightness_range = [0.7 , 1.3], #Tuple of floats; range to pick a brightness value from.
                                       shear_range=0.3,
                                       zoom_range=0.7, # zoom range = [1-zoom_range , 1 + zoom_range]
                                       channel_shift_range = 100,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode = 'nearest' )
    #Fit the augmentation
    #train_datagen.fit()
    
    test_datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = train_datagen.flow_from_directory(directory= train_dir,
                                                        class_mode='categorical',
                                                        target_size=train_input_shape[0:2],
                                                        batch_size=batch_size,
                                                        subset="training",
                                                        shuffle=True,
                                                        classes=artist_names
                                                       )

    valid_generator = train_datagen.flow_from_directory(directory= train_dir,
                                                        class_mode='categorical',
                                                        target_size=train_input_shape[0:2],
                                                        batch_size=batch_size,
                                                        subset="validation",
                                                        shuffle=True,
                                                        classes=artist_names)
    
    test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                      class_mode = 'categorical',
                                                      target_size = train_input_shape[0:2],
                                                      batch_size = 1,
                                                      classes = artist_names)
    
    STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
    print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Found 6862 images belonging to 50 classes.
Found 1186 images belonging to 50 classes.
Found 398 images belonging to 50 classes.
Total number of batches = 428 and 74


In [18]:
import sys
sys.path.insert(1,os.getcwd())

from keras__applications.keras_applications.efficientnet import *

In [19]:
with my_strategy.scope():
    # Load pre-trained model
    base_model = EfficientNetB7(include_top= False,
                                input_shape = train_input_shape,
                                weights = 'imagenet',
                               backend =tf.keras.backend,
                               layers = tf.keras.layers,
                               utils = tf.keras.utils,
                               models = tf.keras.models)

    base_model.summary()

Model: "efficientnet-b7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 600, 600, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 601, 601, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 300, 300, 64) 1728        stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 300, 300, 64) 256         stem_conv[0][0]                  
____________________________________________________________________________________

Total params: 64,097,680
Trainable params: 63,786,960
Non-trainable params: 310,720
__________________________________________________________________________________________________


In [20]:
with my_strategy.scope():
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = tf.keras.layers.Dense(n_classes , activation='softmax')(avg)

    model = tf.keras.Model(inputs = base_model.input, outputs = output)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.2), 
                  metrics=['accuracy'])


In [21]:
with my_strategy.scope():
    #1.callback Tensorboard
    try:
        os.mkdir("my_logs")
    except:
        pass

    root_logdir = os.path.join(os.curdir , "my_logs")

    def get_run_logdir():
        import time
        run_id = time.strftime("Artists_EfficientNet_%Y_%m_%d-%H_%M_%S")
        return os.path.join(root_logdir,run_id)

    run_logdir = get_run_logdir()

    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

    #2.callback EarlyStopping
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                         patience=13 ,
                                                         verbose=1, 
                                                         mode='auto',
                                                         restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.3,
                                  patience=3, 
                                  verbose=1,
                                  mode='auto')
    
    
    # Include the epoch in the file name (uses `str.format`)
    try:
        os.mkdir("checkpoint/")
    except:
        pass
    
    checkpoint_path = "checkpoint/DeepArtist_Efficient_cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights every 5 epochs
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                     verbose=1, 
                                                     save_weights_only=True,
                                                     save_freq = 5)


In [16]:
n_epoch = 10
with my_strategy.scope():

    history_1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb, cp_callback],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 53 steps, validate for 9 steps
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
53/53 [==============================] - 871s 16s/step - loss: 719.2691 - accuracy: 0.0944 - val_loss: 121.7538 - val_accuracy: 0.2144
Epoch 2/10
53/53 [==============================] - 757s 14s/step - loss: 318.6192 - accuracy: 0.1913 - val_loss: 162.0526 - val_accuracy: 0.2526
Epoch 3/10
53/53 [==============================] - 758s 14s/step - loss: 267.8905 - accuracy: 0.2242 - val_loss: 131.1874 - val_accuracy: 0.2726
Epoch 4/10
53/53 [==============================] - 757s 14s/step - loss: 276.4034 - accuracy: 0.2333 - val_loss: 120.6249 - val_accuracy: 0.3307
Epoch 5/10
53/53 [=====

In [12]:
with my_strategy.scope():
    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=10))
    model.load_weights(checkpoint_path.format(epoch=10))

In [26]:
for layer in base_model.layers:
    if 'top' in layer.name:
        layer.trainable = True
    #if layer.trainable:
    #    print(layer.name , "is trainable!")

In [48]:
n_epoch = 10
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.001), # reduce the lr from 0.1 to 0.001
                  metrics=['accuracy'])
    
    history_2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 53 steps, validate for 9 steps
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
53/53 [==============================] - 870s 16s/step - loss: 102.9106 - accuracy: 0.3439 - val_loss: 91.4463 - val_accuracy: 0.4392
Epoch 2/10
53/53 [==============================] - 759s 14s/step - loss: 74.2895 - accuracy: 0.3987 - val_loss: 69.9300 - val_accuracy: 0.4740
Epoch 3/10
53/53 [==============================] - 761s 14s/step - loss: 62.8792 - accuracy: 0.4076 - val_loss: 63.2924 - val_accuracy: 0.4540
Epoch 4/10
53/53 [==============================] - 758s 14s/step - loss: 58.1827 - accuracy: 0.4246 - val_loss: 62.0097 - val_accuracy: 0.4071
Epoch 5/10
53/53 [==============================] - 760s 14s/step - loss: 54.3062 - accuracy: 0.4302 - val_loss: 57.2689 - val_accuracy: 0.4453
Epoch 6/10
53/53 [===================

In [14]:
with my_strategy.scope():
    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=20))
    model.load_weights(checkpoint_path.format(epoch=20))

In [10]:
for layer in base_model.layers:
    if 'block7' in layer.name:
        layer.trainable = True
    
    #if layer.trainable:
    #    print(layer.name , "is trainable!")
        

In [63]:
n_epoch = 30
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.0001), # reduce the lr from 0.1 to 0.001
                  metrics=['accuracy'])
    
    history_3 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 53 steps, validate for 9 steps
Epoch 1/30
53/53 [==============================] - 909s 17s/step - loss: 38.2921 - accuracy: 0.4073 - val_loss: 35.3211 - val_accuracy: 0.5017
Epoch 2/30
53/53 [==============================] - 761s 14s/step - loss: 25.5615 - accuracy: 0.4301 - val_loss: 29.6558 - val_accuracy: 0.4991
Epoch 3/30
53/53 [==============================] - 767s 14s/step - loss: 20.7765 - accuracy: 0.4609 - val_loss: 23.7631 - val_accuracy: 0.5208
Epoch 4/30
53/53 [==============================] - 763s 14s/step - loss: 17.2977 - accuracy: 0.4846 - val_loss: 20.3083 - val_accuracy: 0.4653
Epoch 5/30
53/53 [==============================] - 763s 14s/step - loss: 15.5372 - accuracy: 0.4927 - val_loss: 18.7141 - val_accuracy: 0.5304
Epoch 6/30
53/53 [==============================] - 761s 14s/step - loss: 13.7063 - accuracy: 0.5046 - val_loss: 17.9782 - val_accuracy: 0.5269
Epoch 7/30
53/53 [===========================

In [16]:
with my_strategy.scope():

    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=40))
    model.load_weights(checkpoint_path.format(epoch=40))

In [14]:
n_epoch = 30
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.00005), # reduce the lr from 0.1 to 0.0005
                  metrics=['accuracy'])
    
    history_4 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 53 steps, validate for 9 steps
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 57 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 57 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
53/53 [==============================] - 895s 17s/step - loss: 7.1957 - accuracy: 0.5974 - val_loss: 11.5873 - val_accuracy: 0.5304
Epoch 2/30
53/53 [==============================] - 763s 14s/step - loss: 7.2712 - accuracy: 0.6031 - val_loss: 11.1546 - val_accuracy: 0.5373
Epoch 3/30
53/53 [==============================] - 763s 14s/step - loss: 7.0947 - accuracy: 0.6087 - val_loss: 10.8908 - val_accuracy: 0.5130
Epoch 4/30
53/53 [==============================] - 762s 14s/step - loss: 6.5908 - accuracy: 0.6093 

In [23]:
with my_strategy.scope():

    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=47))
    model.load_weights(checkpoint_path.format(epoch=47))

In [27]:
for layer in base_model.layers:
    if 'block6' in layer.name:
        layer.trainable = True
    
    #if layer.trainable:
    #    print(layer.name , "is trainable!")
        

In [25]:

n_epoch = 30
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.00001), # reduce the lr from 0.1 to 0.0005
                  metrics=['accuracy'])
    
    history_5 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 214 steps, validate for 37 steps
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
214/214 [==============================] - 1206s 6s/step - loss: 11.5400 - accuracy: 0.4982 - val_loss: 11.5402 - val_accuracy: 0.5177
Epoch 2/30
214/214 [==============================] - 884s 4s/step - loss: 10.3421 - accuracy: 0.4999 - val_loss: 11.0236 - val_accuracy: 0.5144
Epoch 3/30
214/214 [==============================] - 891s 4s/step - loss: 9.4719 - accuracy: 0.5193 - val_loss: 10.7728 - val_accuracy: 0.5135
Epoch 4/30
214/214 [==============================] - 891s 4s/step - loss: 9.2350 - accuracy: 0.5277 - val_loss: 10.6949 - val_accuracy: 0.5186
Epoch 5/30
214/214 [==

In [28]:
n_epoch = 50
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.00001), # reduce the lr from 0.1 to 0.000001
                  metrics=['accuracy'])
    
    history_6 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 214 steps, validate for 37 steps
Epoch 1/50
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
214/214 [==============================] - 1091s 5s/step - loss: 8.6219 - accuracy: 0.5451 - val_loss: 11.1337 - val_accuracy: 0.5245
Epoch 2/50
214/214 [==============================] - 888s 4s/step - loss: 8.0030 - accuracy: 0.5501 - val_loss: 10.2519 - val_accuracy: 0.5405
Epoch 3/50
214/214 [==============================] - 882s 4s/step - loss: 7.9934 - accuracy: 0.5600 - val_loss: 10.5764 - val_accuracy: 0.5160
Epoch 4/50
214/214 [==============================] - 883s 4s/step - loss: 7.7308 - accuracy: 0.5622 - val_loss: 10.3852 - val_accuracy: 0.5262
Epoch 5/50
214/214 [====

In [17]:
with my_strategy.scope():

    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=61))
    model.load_weights(checkpoint_path.format(epoch=61))

In [19]:
n_epoch = 10
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.0001), # raise lr a bit
                  metrics=['accuracy'])
    
    history_7 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 214 steps, validate for 37 steps
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 226 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
214/214 [==============================] - 1100s 5s/step - loss: 9.4659 - accuracy: 0.5337 - val_loss: 9.5219 - val_accuracy: 0.5752
Epoch 2/10
214/214 [==============================] - 886s 4s/step - loss: 8.1164 - accuracy: 0.5640 - val_loss: 9.4918 - val_accuracy: 0.6039
Epoch 3/10
214/214 [==============================] - 892s 4s/step - loss: 7.9197 - accuracy: 0.5827 - val_loss: 10.1296 - val_accuracy: 0.5718
Epoch 4/10
214/214 [==============================] - 891s 4s/step - loss: 6.7602 - accuracy: 0.6165 - val_loss: 8.7930 - val_accuracy: 0.6073
Epoch 5/10
214/214 [=======

In [35]:
with my_strategy.scope():

    # Save or load the weights using the `checkpoint_path` format
    #model.save_weights(checkpoint_path.format(epoch=71))
    model.load_weights(checkpoint_path.format(epoch=71))

In [36]:
#Let the model trainable from 'top' to 'block5'
for layer in base_model.layers:

    if 'block4' in layer.name:
        layer.trainable = False
   
    if 'block5' in layer.name:
        layer.trainable = True
    
    if layer.trainable:
        print(layer.name , "is trainable!")

  

block5a_expand_conv is trainable!
block5a_expand_bn is trainable!
block5a_expand_activation is trainable!
block5a_dwconv is trainable!
block5a_bn is trainable!
block5a_activation is trainable!
block5a_se_squeeze is trainable!
block5a_se_reshape is trainable!
block5a_se_reduce is trainable!
block5a_se_expand is trainable!
block5a_se_excite is trainable!
block5a_project_conv is trainable!
block5a_project_bn is trainable!
block5b_expand_conv is trainable!
block5b_expand_bn is trainable!
block5b_expand_activation is trainable!
block5b_dwconv is trainable!
block5b_bn is trainable!
block5b_activation is trainable!
block5b_se_squeeze is trainable!
block5b_se_reshape is trainable!
block5b_se_reduce is trainable!
block5b_se_expand is trainable!
block5b_se_excite is trainable!
block5b_project_conv is trainable!
block5b_project_bn is trainable!
block5b_drop is trainable!
block5b_add is trainable!
block5c_expand_conv is trainable!
block5c_expand_bn is trainable!
block5c_expand_activation is traina

In [37]:
n_epoch = 15
with my_strategy.scope():
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Nadam(learning_rate = 0.0001), # raise lr a bit
                  metrics=['accuracy'])
    
    history_8 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                          validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                          epochs=n_epoch,
                          shuffle=True,
                          verbose=1,
                          callbacks=[lr_scheduler,early_stopping_cb,tensorboard_cb],
                          class_weight=CLASS_WEIGHT
                         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 428 steps, validate for 74 steps
Epoch 1/15
INFO:tensorflow:batch_all_reduce: 304 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 304 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
428/428 [==============================] - 1150s 3s/step - loss: 8.1210 - accuracy: 0.5787 - val_loss: 7.8577 - val_accuracy: 0.6503
Epoch 2/15
428/428 [==============================] - 903s 2s/step - loss: 7.1473 - accuracy: 0.6049 - val_loss: 8.3195 - val_accuracy: 0.6123
Epoch 3/15
428/428 [==============================] - 897s 2s/step - loss: 5.8066 - accuracy: 0.6382 - val_loss: 7.2092 - val_accuracy: 0.6402
Epoch 4/15
428/428 [==============================] - 914s 2s/step - loss: 5.6532 - accuracy: 0.6478 - val_loss: 7.7305 - val_accuracy: 0.6562
Epoch 5/15
428/428 [========

In [38]:
with my_strategy.scope():

    # Save or load the weights using the `checkpoint_path` format
    model.save_weights(checkpoint_path.format(epoch=86))
    #model.load_weights(checkpoint_path.format(epoch=86))

In [40]:
with my_strategy.scope():

    test_datagen = ImageDataGenerator(rescale = 1./255)
    test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                  class_mode = 'categorical',
                                                  target_size = train_input_shape[0:2],
                                                  batch_size = 1,
                                                  classes = artist_names)
    STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

    score = model.evaluate_generator(test_generator, STEP_SIZE_TEST)
    
    print("Prediction [Loss , accuracy] on test data =" , score)

Found 398 images belonging to 50 classes.
Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']
Prediction [Loss , accuracy] on test data = [0.26394664083264796, 0.7788945]


In [ ]:

class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, patience = 5):
        super(MyCallback, self).__init__()
        self.patience = patience
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf
    def on_epoch_end(self, epoch , logs = {}):
        current = logs.get('loss')
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('Restoring model weights from the end of the best epoch.')
                self.model.set_weights(self.best_weights)
         
        if logs.get('val_acc') < log.get('acc') :
                if logs.get('val_acc') < best_val_acc :
                    self.patience -= 1
                    if self.patience == 0:
                        self.model.stop_training = True

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))